In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne 

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
import openai
openai.api_key = "sk-xxx"

In [ ]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader

loader=YoutubeTranscriptReader()

youtube_documents = loader.load_data(ytlinks=['https://www.youtube.com/watch?v=jITPOcBQQW8', 'https://www.youtube.com/watch?v=xFfnJhZeL_Y', 'https://www.youtube.com/watch?v=g_LNu6Aaxvk'])
print(youtube_documents)

In [7]:
from llama_index import VectorStoreIndex

vector_indices = {}
vector_query_engines = {}

breeds = ["Savanah", "Ragdol", "Maine Coon"]

for breed, youtube in zip(breeds, youtube_documents):
    vector_index = VectorStoreIndex.from_documents([youtube])
    query_engine = vector_index.as_query_engine(similarity_top_k=3)
    vector_indices[breed] = vector_index
    vector_query_engines[breed] = query_engine

In [8]:
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata


query_engine_tools = []
for breed in breeds:
    query_engine = vector_query_engines[breed]

    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=breed,
            description=f"Provides information about the cat breed {breed}"
        ),
    )
    query_engine_tools.append(query_engine_tool)

subquestion_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)    

In [9]:
response = subquestion_engine.query("Tell me the difference between Ragdoll and Maine Coon cat.")

Generated 6 sub questions.
[Ragdol] Q: What are the characteristics of Ragdoll cats?
[Maine Coon] Q: What are the characteristics of Maine Coon cats?
[Ragdol] Q: What is the size of Ragdoll cats?
[Maine Coon] Q: What is the size of Maine Coon cats?
[Ragdol] Q: What is the temperament of Ragdoll cats?
[Maine Coon] Q: What is the temperament of Maine Coon cats?
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=33 request_id=d35ce30758c21ce2ceb50326bdc6d6b0 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=33 request_id=d35ce30758c21ce2ceb50326bdc6d6b0 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=53 request_id=5c49044334c0db4640457afe390e0fbe response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=53 request_id=5c49044334c0db4640457afe390e0fbe response_code=200
INFO:opena

In [10]:
print(response)

Ragdoll cats are known for their large size, intelligence, affectionate nature, and ability to articulate their needs. They are intuitive when it comes to showing affection and companionship. Ragdolls are also trainable and can be taught to play fetch, chase a toy, or even ring a bell. They are considered one of the most affectionate cat breeds and love to cuddle and be close to their humans. Ragdolls are often described as "floppy cats" because they have a genetic trait that makes them go limp when cradled in your arms. They are generally friendly, gentle, and tolerant with people and other animals, making them great pets for families with children and other pets. Ragdolls can be quite vocal and talkative, using their meows to communicate their needs or desires. However, they are not prone to aggression and are generally well-behaved.

On the other hand, Maine Coon cats are known for their large size, with males weighing up to 25 pounds and females weighing between ten to fourteen pou